In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm

from scipy.stats import gamma
from scipy.stats import norm
from scipy.stats import multivariate_normal
from scipy.stats import gaussian_kde
from scipy.special import expit
from scipy.special import logsumexp
from scipy.optimize import minimize
from scipy.optimize import LinearConstraint
from scipy.signal import convolve
from scipy.interpolate import interp1d

import divebomb

import time
import pickle

import importlib

np.random.seed(0)

In [2]:
import Preprocessor
import Parameters
import HHMM
import Visualisor

In [3]:
pars = Parameters.Parameters()

pars.features = [{'dive_duration':{'corr':False,'f':'gamma'}},
                 {'FoVeDBA_low':{'thresh':5,'corr':True,'f':'gamma'}}]#,
                  #'Vz':{'corr':True,'f':'normal'}}]

prep = Preprocessor.Preprocessor(pars)

In [4]:
np.set_printoptions(suppress=True,precision=4)

In [5]:
s_dive = 130
e_dive = 200
pars.K[0] = 3
pars.K[1] = 3
pars.share_fine_states = True
HHMM_file = 'HHMM_k_%s_%s_dives_%s_%s_same_fine_states' % (pars.K[0],pars.K[1],s_dive,e_dive)
print(HHMM_file)

HHMM_k_3_3_dives_130_200_same_fine_states


# Preprocess Data

In [ ]:
df = prep.load_data(pars.cvc_file,
                    pars.csv_file,
                    pars.cvc_cols)

df = prep.prune_cols(df)

df = prep.prune_times(df,
                      pars.stime,
                      pars.etime,
                      pars.drop_times)

df = prep.fix_pressure(df)

df = prep.find_Vz(df)

df = prep.smooth_columns(df,
                         pars.smoother,
                         pars.smooth_cols)

df,dive_df = prep.find_dives(df)

data = prep.get_all_features(df,dive_df)

viz = Visualisor.Visualisor(pars,data,df)

In [ ]:
viz.lagplot(lims={'peak_jerk':[0,50],
                  'roll_at_pj':[0,1],
                  'heading_var':[0,0.2],
                  'FoVeDBA_low':[0,5000],
                  'FoVeDBA_high':[0,10],
                  'Vz':[-5,5]})

# Train HHMM

In [ ]:
deep_data = [data[i] for i in range(s_dive,e_dive)]
importlib.reload(HHMM)
hhmm = HHMM.HHMM(pars,deep_data)
print(hhmm.likelihood(deep_data))

# initial crude theta
hhmm.theta[0]['dive_duration']['mu'][0] = 200
hhmm.theta[0]['dive_duration']['sig'][0] = 100

hhmm.theta[0]['dive_duration']['mu'][1] = 40
hhmm.theta[0]['dive_duration']['sig'][1] = 20

hhmm.theta[0]['dive_duration']['mu'][2] = 30
hhmm.theta[0]['dive_duration']['sig'][2] = 15

# inital fine theta
#hhmm.theta[1][0]['FoVeDBA_low']['mu'][1] = 10
#hhmm.theta[1][0]['FoVeDBA_low']['sig'][1] = 50
#hhmm.theta[1][0]['FoVeDBA_low']['mu'][0] = 250
#hhmm.theta[1][0]['FoVeDBA_low']['sig'][0] = 250

#hhmm.theta[1][1]['FoVeDBA_low']['mu'][0] = 10
#hhmm.theta[1][1]['FoVeDBA_low']['sig'][0] = 50
#hhmm.theta[1][1]['FoVeDBA_low']['mu'][1] = 400
#hhmm.theta[1][1]['FoVeDBA_low']['sig'][1] = 1000

#hhmm.theta[1][2]['FoVeDBA_low']['mu'][1] = 10
#hhmm.theta[1][2]['FoVeDBA_low']['sig'][1] = 50
#hhmm.theta[1][2]['FoVeDBA_low']['mu'][0] = 250
#hhmm.theta[1][2]['FoVeDBA_low']['sig'][0] = 250

try:
    hhmm = hhmm.load(HHMM_file)
except:
    print('HHMM not found. Training new model')
    hhmm.train_DM(deep_data)
    hhmm.save(HHMM_file)
    
data,df = hhmm.label_df(data,df)
print(hhmm.likelihood(deep_data))

# Visualize Results

In [ ]:
importlib.reload(Visualisor)
viz = Visualisor.Visualisor(pars,data,df)
viz.hhmm = hhmm
viz.plot_emission_probs(0)
viz.plot_emission_probs(1)
viz.print_ptms()
viz.plot_dive_features(130,200,['depth','VeDBA','subdive_state_1_prob','dive_state_2_prob'],
                               ['FoVeDBA_low'])

# Things to note:

    - Subdive states are much less sure than dive types
    - Subdive states are more uncertain in shallow dives than deep dives
        - should probably make different dstribtuions within different dive types
    - VeDBA is very different within shallow dives compared to deep dives
        - support for activity-specific use of energy readings

# Future Steps:

    - Include other features (high-frequency VeDBA, roll, heading variance, peak jerk, etc.?)
    - Use different subdive behaviors for different dive types
    - Figure out what number of states in each level
    - Write paper on fine-scale HHMMs and equivalence between correlated HMMs and continuous time models